<a href="https://colab.research.google.com/github/dagmaros27/AIMS_Notebooks/blob/main/CUDA_Practical_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **CUDA Programming on NVIDIA GPUs, July 22-26, 2024**

# **Practical 1**

First of all, make sure the correct Runtime is being used, by clicking on the Runtime option at the top, then "Change runtime type", selecting an appropriate GPU such as the T4, then clicking Save.

A Colab Pro or Pro+ account will allow you to use a more powerful GPU, but the freely available T4 is perfectly adequate for the practicals in this course. It has good single precision capabilities and corresponds to Compute Capability 7.5.

To check that this has been done successfully, the first instruction below returns information on the version of the available NVIDIA compiler, and the second instruction returns information on the GPU which is available to you.  

To "execute" each cell, click on the little triangle to the left of the instructions.  The ! tells Colab that these are system commands to be executed.

In [ ]:
!nvcc --version


nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2024 NVIDIA Corporation
Built on Thu_Jun__6_02:18:23_PDT_2024
Cuda compilation tools, release 12.5, V12.5.82
Build cuda_12.5.r12.5/compiler.34385749_0


In [ ]:
!nvidia-smi

Tue Jan 13 09:13:08 2026       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   48C    P8              9W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

---

The first step is to upload two header files from the course webpage.

In [ ]:
!wget https://people.maths.ox.ac.uk/gilesm/cuda/headers/helper_cuda.h
!wget https://people.maths.ox.ac.uk/gilesm/cuda/headers/helper_string.h

--2026-01-13 09:13:31--  https://people.maths.ox.ac.uk/gilesm/cuda/headers/helper_cuda.h
Resolving people.maths.ox.ac.uk (people.maths.ox.ac.uk)... 129.67.184.129, 2001:630:441:201::8143:b881
Connecting to people.maths.ox.ac.uk (people.maths.ox.ac.uk)|129.67.184.129|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 27832 (27K) [text/x-chdr]
Saving to: ‘helper_cuda.h’

helper_cuda.h       100%[===================>]  27.18K   159KB/s    in 0.2s    

2026-01-13 09:13:33 (159 KB/s) - ‘helper_cuda.h’ saved [27832/27832]

--2026-01-13 09:13:33--  https://people.maths.ox.ac.uk/gilesm/cuda/headers/helper_string.h
Resolving people.maths.ox.ac.uk (people.maths.ox.ac.uk)... 129.67.184.129, 2001:630:441:201::8143:b881
Connecting to people.maths.ox.ac.uk (people.maths.ox.ac.uk)|129.67.184.129|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 14875 (15K) [text/x-chdr]
Saving to: ‘helper_string.h’

helper_string.h     100%[===================>]  14.53K 


---
Next we create the file prac1a.cu by using the %%writefile instruction at the top of the code block.

In doing this, we are following the helpful information provided here:
https://colab.research.google.com/drive/1GJOfTp56OeQRdE4u2_S7pUNRcJb4ik9X?usp=sharing

In [ ]:
%%writefile prac1a.cu

//
// include files
//

#include <stdlib.h>
#include <stdio.h>
#include <string.h>
#include <math.h>

//
// kernel routine
//

__global__ void my_first_kernel(float *x)
{
  int tid = threadIdx.x + blockDim.x*blockIdx.x;

  x[tid] = (float) threadIdx.x;
}


//
// main code
//

int main(int argc, const char **argv)
{
  float *h_x, *d_x;
  int   nblocks, nthreads, nsize, n;

  // initialise card

  findCudaDevice(argc, argv);

  // set number of blocks, and threads per block

  nblocks  = 2;
  nthreads = 8;
  nsize    = nblocks*nthreads;

  // allocate memory for array

  h_x = (float *) malloc(nsize*sizeof(float));
  cudaMalloc((void **) &d_x, nsize*sizeof(float));

  // execute kernel

  my_first_kernel<<<nblocks,nthreads>>>(d_x);

  // copy back results and print them out

  cudaMemcpy(h_x,d_x,nsize*sizeof(float),cudaMemcpyDefault);

  for (n=0; n<nsize; n++) printf(" n,  x  =  %d  %f \n",n,h_x[n]);

  // free memory

  cudaFree(d_x);
  free(h_x);

  // CUDA exit -- needed to flush printf write buffer

  cudaDeviceReset();

  return 0;
}


Writing prac1a.cu




---

We use the following instruction to compile prac1a.cu to create the executable output prac1a.  The other flags are as follows:

-I. says to look in the current directory for header files

-lineinfo helps with debugging if there's a run-time problem

-arch=sm_70 says it is for GPUs of Compute Capability 7.0 or later

--ptxas=-v gives us additional information such as how many registers are used

--use_fast_math generates faster code which might sometimes be a little less accurate

-lcudart links in the run-time CUDA library



In [ ]:
!nvcc prac1a.cu -o prac1a -I. -lineinfo -arch=sm_70 --ptxas-options=-v --use_fast_math -lcudart

ptxas info    : 0 bytes gmem
ptxas info    : Compiling entry function '_Z15my_first_kernelPf' for 'sm_70'
ptxas info    : Function properties for _Z15my_first_kernelPf
    0 bytes stack frame, 0 bytes spill stores, 0 bytes spill loads
ptxas info    : Used 10 registers, 360 bytes cmem[0]


---

Now we can execute the code.


In [ ]:
!./prac1a

 n,  x  =  0  0.000000 
 n,  x  =  1  1.000000 
 n,  x  =  2  2.000000 
 n,  x  =  3  3.000000 
 n,  x  =  4  4.000000 
 n,  x  =  5  5.000000 
 n,  x  =  6  6.000000 
 n,  x  =  7  7.000000 
 n,  x  =  8  0.000000 
 n,  x  =  9  1.000000 
 n,  x  =  10  2.000000 
 n,  x  =  11  3.000000 
 n,  x  =  12  4.000000 
 n,  x  =  13  5.000000 
 n,  x  =  14  6.000000 
 n,  x  =  15  7.000000 




---

We can now perform the same steps for the second code, prac1b.cu, which does lots of error-checking.

In [ ]:
%%writefile prac1b.cu

//
// include files
//

#include <stdlib.h>
#include <stdio.h>
#include <string.h>
#include <math.h>

#include <helper_cuda.h>


//
// kernel routine
//

__global__ void my_first_kernel(float *x)
{
  int tid = threadIdx.x + blockDim.x*blockIdx.x;

  x[tid] = (float) threadIdx.x;
}


//
// main code
//

int main(int argc, const char **argv)
{
  float *h_x, *d_x;
  int   nblocks, nthreads, nsize, n;

  // initialise card

  findCudaDevice(argc, argv);

  // set number of blocks, and threads per block

  nblocks  = 2;
  nthreads = 8;
  nsize    = nblocks*nthreads;

  // allocate memory for array

  h_x = (float *) malloc(nsize*sizeof(float));
  checkCudaErrors(cudaMalloc((void **) &d_x, nsize*sizeof(float)));

  // execute kernel

  my_first_kernel<<<nblocks,nthreads>>>(d_x);
  getLastCudaError("my_first_kernel execution failed\n");

  // copy back results and print them out

  checkCudaErrors( cudaMemcpy(h_x,d_x,nsize*sizeof(float),
                 cudaMemcpyDefault) );

  for (n=0; n<nsize; n++) printf(" n,  x  =  %d  %f \n",n,h_x[n]);

  // free memory

  checkCudaErrors(cudaFree(d_x));
  free(h_x);

  // CUDA exit -- needed to flush printf write buffer

  cudaDeviceReset();

  return 0;
}


Writing prac1b.cu


In [ ]:
!nvcc prac1b.cu -o prac1b -I. -lineinfo -arch=sm_70 --ptxas-options=-v --use_fast_math -lcudart

ptxas info    : 0 bytes gmem
ptxas info    : Compiling entry function '_Z15my_first_kernelPf' for 'sm_70'
ptxas info    : Function properties for _Z15my_first_kernelPf
    0 bytes stack frame, 0 bytes spill stores, 0 bytes spill loads
ptxas info    : Used 10 registers, 360 bytes cmem[0]


In [ ]:
!./prac1b

GPU Device 0: "Turing" with compute capability 7.5

 n,  x  =  0  0.000000 
 n,  x  =  1  1.000000 
 n,  x  =  2  2.000000 
 n,  x  =  3  3.000000 
 n,  x  =  4  4.000000 
 n,  x  =  5  5.000000 
 n,  x  =  6  6.000000 
 n,  x  =  7  7.000000 
 n,  x  =  8  0.000000 
 n,  x  =  9  1.000000 
 n,  x  =  10  2.000000 
 n,  x  =  11  3.000000 
 n,  x  =  12  4.000000 
 n,  x  =  13  5.000000 
 n,  x  =  14  6.000000 
 n,  x  =  15  7.000000 




---
By going back to the previous code blocks you can modify the codes to complete the Practical 1 exercises.  Alternatively you can copy a Code cell (on my system I do this by using the mouse right-click) and paste it (control-V on my system) to form a new Code cell -- this is best for the final exercise in which you are to write a new code to add two vectors.

However, this copy of the notebook is read-only for everyone except the owner (me!) so you will need to make your own copy of the notebook by going to the File option at the top and then clicking on "Save a copy in Drive" which will make a copy of it in your Google Drive.  You are then the owner of the copy and can edit it freely.

For students doing this as an assignment to be assessed, you should add your name to the title of the notebook (as in "Practical 1 -- Mike Giles.ipynb"), make it shared (see the Share option in the top-right corner) and provide the shared link as the submission mechanism.

This final piece of code terminates the runtime for this notebook so that you can switch to a new notebook without any problems -- you will get an error message if you try to keep two runtimes going at the same time with the free Colab account.

It's particularly convenient if you are executing the whole notebook to check everything works correctly, using the "Run all" option in the Runtime tab.

In [ ]:
from google.colab import runtime
runtime.unassign()